# Carga de librerias

In [1]:
from pymongo import MongoClient
from json import dumps
import numpy as np
import os

client = MongoClient('localhost', 27017)
db = client[f"test_mongo"]

# Inserción de 1 millón de documentos en campos ya indexados

En esta sección se revisará la carga y el tamaño de almacenamiento de 1 millón de documentos indexados en la base de datos.

## Carga de documento

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-mongo.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-mongo.py -n 1

```

El script antes mencionado recibe como entrada la cantidad de documentos que cargará a la base de datos. Secuencialmente, el código creará la nueva colección donde se cargarán los datos, creará los índices correspondientes a todos los campos y cargará la información almacenada en un documento JSON de forma masiva.

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/mongo-insert/load_1M_documents_mongo.png)

Como se puede ver en la imagen superior, cargar 1 millón de documentos sobre campos ya indexados ha tomado un tiempo de **1 minuto y 1.51 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en la colección ```extracto_cuenta_1m``` de la siguiente manera.

In [116]:
cnt = db.extracto_cuenta_1m.count_documents({})
print(f"Actualmente existen {cnt} documentos en la colección 'extracto_cuenta_1m'")

Actualmente existen 1000000 documentos en la colección 'extracto_cuenta_1m'


## Tamaño de almacenamiento de datos

Para encontrar el tamaño de los archivos JSON en su forma bruta, antes de cargarse a la base de datos, se utilizará el siguiente script:

In [124]:
file_sizes = []
n = 1

for i, json_file in enumerate(sorted(os.listdir("../json-generated-data"))):
    file_stat = os.stat(os.path.join("../json-generated-data", json_file))
    file_sizes.append(file_stat)

    if i+1 == n:
        break

for file in file_sizes:
    print(f'Documento {json_file} con tamaño en memoria de {file.st_size} bytes')

Documento random-generated-data-1.json con tamaño en memoria de 259734499 bytes


Para obtener las estadísticas de uso de disco por parte de la colección ```extracto_cuenta_1m``` en la base de datos ejecutaremos el siguiente comando:

In [5]:
r = dumps(db.command(
   {
     "collStats": "extracto_cuenta_1m",
     "scale": 1
   }
), indent=1)

print(r[:300])
print("...")


{
 "ns": "test_mongo.extracto_cuenta_1m",
 "size": 206123814,
 "count": 1000000,
 "avgObjSize": 206,
 "numOrphanDocs": 0,
 "storageSize": 103682048,
 "freeStorageSize": 409600,
 "capped": false,
 "wiredTiger": {
  "metadata": {
   "formatVersion": 1
  },
  "creationString": "access_pattern_hint=none
...


Extrayendo información importante de los datos recién obtenidos. El tamaño de almacenamiento aproximado de 1 documento de 1 millón de registros con un peso en bruto de 247.70 mb (en escala base binario $2^{20}$), una vez comprimido en la base de datos es **98.87 mb** (en escala base binario $2^{20}$). Esto nos da un radio de compresión de 0.3991 y una reducción de tamaño en memoria del 60.08%.

![Cargando 1 millón de documentos](../imagenes-de-soporte/mongo-insert/storage_compass_1M_documents.png)

Mediante la herramienta gráfica compass se puede ver un reporte de 103.27 MB (en escala base decimal $10^{6}$) de tamaño de almacenamiento, lo cual convertido a escala binaria resulta en 98.23 mb, un número aproximado al obtenido mediante el script de python.

In [ ]:
print("...")
print(r[109825:])

...
"indexBuilds": [],
 "totalIndexSize": 243048448,
 "totalSize": 346730496,
 "indexSizes": {
  "_id_": 11104256,
  "fecha_1": 41160704,
  "agencia_1": 6668288,
  "monto_1": 34279424,
  "saldo_1": 36990976,
  "descripcion_1": 6410240,
  "nota_1": 106434560
 },
 "scaleFactor": 1,
 "ok": 1.0
}


Otro número importante que se meustra en compass es el tamaño total de los índices (7 índices en este caso), siendo 243.05 MB (en escala base decimal $10^{6}$) o según los datos extraídos mediante python **231.79 mb** (en escala base binario $2^{20}$).

Una última métrica importante ha revisar es la suma del espacio asignado tanto a los documentos como a los índices en la colección. Es la suma de storageSize y indexSize. En el presente caso resultando un valor de **330.67 mb**.



| Tabla resumen de resultados                 | Valor obtenido |
|---------------------------------------------|----------------|
| Tiempo de carga                             | 1 min 1.52 s   |
| Tamaño JSON bruto                           | 247.70 mb      |
| Tamaño de almacenamiento en mongoDB         | 98.87 mb       |
| Radio de compresión de documento            | 0.3991         |
| Reducción de tamaño de memoria de documento | 60.08 %        |
| Tamaño total de los índices                 | 231.79 mb      |
| Tamaño total de almacenamiento colección    | 330.67 mb      |

# Inserción de 10 millones de documentos en campos ya indexados

En esta sección se revisará la carga y el tamaño de almacenamiento de 10 millones de documentos indexados en la base de datos.

## Carga de documento

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-mongo.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-mongo.py -n 10

```

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/mongo-insert/load_10M_indexed_documents_mongo.png)

Como se puede ver en la imagen superior, cargar 10 millón de documentos sobre campos ya indexados ha tomado un tiempo de **3 horas 12 minutos y 37.32 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en la colección ```extracto_cuenta_10m``` de la siguiente manera.

In [6]:
cnt = db.extracto_cuenta_10m.count_documents({})
print(f"Actualmente existen {cnt} documentos en 'extracto_cuenta_10m'")

Actualmente existen 10000000 documentos en 'extracto_cuenta_10m'


## Tamaño de almacenamiento de datos

Para encontrar el tamaño de los archivos JSON en su forma bruta, antes de cargarse a la base de datos, se utilizará el siguiente script:

In [17]:
file_sizes = []
file_names = []
sum = 0

n = 10

for i, json_file in enumerate(sorted(os.listdir("../json-generated-data"))):
    file_stat = os.stat(os.path.join("../json-generated-data", json_file))
    file_sizes.append(file_stat)
    file_names.append(json_file)
    sum = sum + int(file_stat.st_size)
    
    if i+1 == n:
        break

for file, f_name in zip(file_sizes, file_names):
    print(f'Documento {f_name} con tamaño en memoria de {file.st_size} bytes')

print("")
print(f'Sumando un total de {sum} bytes')

Documento random-generated-data-1.json con tamaño en memoria de 259734499 bytes
Documento random-generated-data-10.json con tamaño en memoria de 259687666 bytes
Documento random-generated-data-11.json con tamaño en memoria de 259697531 bytes
Documento random-generated-data-12.json con tamaño en memoria de 259660386 bytes
Documento random-generated-data-13.json con tamaño en memoria de 259651170 bytes
Documento random-generated-data-14.json con tamaño en memoria de 259696780 bytes
Documento random-generated-data-15.json con tamaño en memoria de 259771941 bytes
Documento random-generated-data-16.json con tamaño en memoria de 259765264 bytes
Documento random-generated-data-17.json con tamaño en memoria de 259736398 bytes
Documento random-generated-data-18.json con tamaño en memoria de 259722533 bytes

Sumando un total de 2597124168 bytes


Para obtener las estadísticas de uso de disco por parte de la colección ```extracto_cuenta_10m``` en la base de datos ejecutaremos el siguiente comando:

In [20]:
r = dumps(db.command(
   {
     "collStats": "extracto_cuenta_10m",
     "scale": 1
   }
), indent=1)

print(r[:300])
print("...")


{
 "ns": "test_mongo.extracto_cuenta_10m",
 "size": 2061015647,
 "count": 10000000,
 "avgObjSize": 206,
 "numOrphanDocs": 0,
 "storageSize": 1043173376,
 "freeStorageSize": 2236416,
 "capped": false,
 "wiredTiger": {
  "metadata": {
   "formatVersion": 1
  },
  "creationString": "access_pattern_hint
...


Extrayendo información importante de los datos recién obtenidos. El tamaño de almacenamiento de 10 documentos de 1 millón de registros con un peso en total en bruto de 2.41 gb (en escala base binario $2^{20}$), una vez comprimido en la base de datos es **0.97 gb** (en escala base binario $2^{20}$). Esto nos da un radio de compresión de 0.4017 y una reducción de tamaño en memoria del 59.83%.

![Cargando 10 millones de documentos](../imagenes-de-soporte/mongo-insert/storage_compass_10M_documents.png)

Mediante la herramienta gráfica compass se puede ver un reporte de 1.04 GB (en escala base decimal $10^{6}$) de tamaño de almacenamiento, lo cual convertido a escala binaria resulta en 0.968 gb, un número aproximado al obtenido mediante el script de python.

In [31]:
print("...")
print(r[109772:])

...
 },
 "indexBuilds": [],
 "totalIndexSize": 3362054144,
 "totalSize": 4405227520,
 "indexSizes": {
  "_id_": 118546432,
  "fecha_1": 601604096,
  "agencia_1": 59777024,
  "monto_1": 514748416,
  "saldo_1": 563314688,
  "descripcion_1": 57679872,
  "nota_1": 1446383616
 },
 "scaleFactor": 1,
 "ok": 1.0
}


Otro número importante que se meustra en compass es el tamaño total de los índices (7 índices en este caso), siendo 3.36 GB (en escala base decimal $10^{6}$) o según los datos extraídos mediante python **3.13 gb** (en escala base binario $2^{20}$).

Una última métrica importante ha revisar es la suma del espacio asignado tanto a los documentos como a los índices en la colección. Es la suma de storageSize y indexSize. En el presente caso resultando un valor de **4.10 gb**.



| Tabla resumen de resultados                 | Valor obtenido       |
|---------------------------------------------|----------------------|
| Tiempo de carga                             | 3 h 12 min y 37.32 s |
| Tamaño JSON bruto                           | 2.41 gb              |
| Tamaño de almacenamiento en mongoDB         | 0.97 gb              |
| Radio de compresión de documento            | 0.4017               |
| Reducción de tamaño de memoria de documento | 59.83 %              |
| Tamaño total de los índices                 | 3.13 gb              |
| Tamaño total de almacenamiento colección    | 4.10 gb              |

# Inserción de 1 millón de documentos y posterior indexación

En esta sección se revisará la carga de 1 millón de documentos en una colección sin índices y el tamaño de almacenamiento de dicha cantidad de documentos una vez ya indexados.

## Carga de documento

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-mongo-no-index.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-mongo-no-index.py -n 1

```

El script antes mencionado recibe como entrada la cantidad de documentos que cargará a la base de datos. Secuencialmente, el código creará la nueva colección donde se cargarán los datos, cargará la información almacenada en un documento JSON de forma masiva, y creará finalmente los índices correspondientes a todos los campos.

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/mongo-insert/load_1M_documentos_no_index_mongo.png)

Como se puede ver en la imagen superior, cargar 1 millón de documentos sobre campos y posteriormente indexar todos los campos ha tomado un tiempo de **35.36 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en la colección ```extracto_cuenta_1m2``` de la siguiente manera.

In [ ]:
cnt = db.extracto_cuenta_1m2.count_documents({})
print(f"Actualmente existen {cnt} documentos en la colección 'extracto_cuenta_1m2'")

Actualmente existen 1000000 documentos en la colección 'extracto_cuenta_1m2'


## Tamaño de almacenamiento de datos

Para obtener las estadísticas de uso de disco por parte de la colección ```extracto_cuenta_1m2``` en la base de datos ejecutaremos el siguiente comando:

In [34]:
r = dumps(db.command(
   {
     "collStats": "extracto_cuenta_1m2",
     "scale": 1
   }
), indent=1)

print(r[:300])
print("...")


{
 "ns": "test_mongo.extracto_cuenta_1m2",
 "size": 206123814,
 "count": 1000000,
 "avgObjSize": 206,
 "numOrphanDocs": 0,
 "storageSize": 100823040,
 "freeStorageSize": 0,
 "capped": false,
 "wiredTiger": {
  "metadata": {
   "formatVersion": 1
  },
  "creationString": "access_pattern_hint=none,all
...


Por los datos recopilados anteriormente se sabe que 1 documento de 1 millón de registros con un peso en bruto de 247.70 mb (en escala base binario $2^{20}$), una vez comprimido en la base de datos, para esta prueba en particular es **96.15 mb** (en escala base binario $2^{20}$). Esto nos da un radio de compresión de 0.3882 y una reducción de tamaño en memoria del 61.18%.

![Cargando 1 millón de documentos](../imagenes-de-soporte/mongo-insert/storage_compass_1M_documentos_no_index.png)

Mediante la herramienta gráfica compass se puede ver un reporte de 100.82 MB (en escala base decimal $10^{6}$) de tamaño de almacenamiento, lo cual convertido a escala binaria resulta en 96.15 mb, un número aproximado al obtenido mediante el script de python.

In [41]:
print("...")
print(r[109788:])

...
 },
 "indexBuilds": [],
 "totalIndexSize": 128323584,
 "totalSize": 229146624,
 "indexSizes": {
  "_id_": 10743808,
  "fecha_1": 20262912,
  "agencia_1": 5373952,
  "monto_1": 16683008,
  "saldo_1": 18006016,
  "descripcion_1": 5603328,
  "nota_1": 51650560
 },
 "scaleFactor": 1,
 "ok": 1.0
}


Otro número importante que se meustra en compass es el tamaño total de los índices (7 índices en este caso), siendo 128.32 MB (en escala base decimal $10^{6}$) o según los datos extraídos mediante python **122.37 mb** (en escala base binario $2^{20}$).

Una última métrica importante ha revisar es la suma del espacio asignado tanto a los documentos como a los índices en la colección. Es la suma de storageSize y indexSize. En el presente caso resultando un valor de **218.53 mb**.



| Tabla resumen de resultados                 | Valor obtenido |
|---------------------------------------------|----------------|
| Tiempo de carga                             | 35.36 s        |
| Tamaño JSON bruto                           | 247.70 mb      |
| Tamaño de almacenamiento en mongoDB         | 96.15 mb       |
| Radio de compresión de documento            | 0.3882         |
| Reducción de tamaño de memoria de documento | 61.18 %        |
| Tamaño total de los índices                 | 122.37 mb      |
| Tamaño total de almacenamiento colección    | 218.53 mb      |

# Inserción de 10 millones de documentos y posterior indexación

En esta sección se revisará la carga de 10 millones de documentos en una colección sin índices y el tamaño de almacenamiento de dicha cantidad de documentos una vez ya indexados.

## Carga de documento

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-mongo-no-index.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-mongo-no-index.py -n 10

```

El script antes mencionado recibe como entrada la cantidad de documentos que cargará a la base de datos. Secuencialmente, el código creará la nueva colección donde se cargarán los datos, cargará la información almacenada en un documento JSON de forma masiva, y creará finalmente los índices correspondientes a todos los campos.

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/mongo-insert/load_10M_documentos_no_index_mongo.png)

Como se puede ver en la imagen superior, cargar 1 millón de documentos sobre campos y posteriormente indexar todos los campos ha tomado un tiempo de **7 minutos 1.82 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en la colección ```extracto_cuenta_10m2``` de la siguiente manera.

In [ ]:
cnt = db.extracto_cuenta_10m2.count_documents({})
print(f"Actualmente existen {cnt} documentos en la colección 'extracto_cuenta_10m2'")

Actualmente existen 1000000 documentos en la colección 'extracto_cuenta_10m2'


## Tamaño de almacenamiento de datos

Para obtener las estadísticas de uso de disco por parte de la colección ```extracto_cuenta_10m2``` en la base de datos ejecutaremos el siguiente comando:

In [43]:
r = dumps(db.command(
   {
     "collStats": "extracto_cuenta_10m2",
     "scale": 1
   }
), indent=1)

print(r[:300])
print("...")


{
 "ns": "test_mongo.extracto_cuenta_10m2",
 "size": 2061015647,
 "count": 10000000,
 "avgObjSize": 206,
 "numOrphanDocs": 0,
 "storageSize": 1004490752,
 "freeStorageSize": 2162688,
 "capped": false,
 "wiredTiger": {
  "metadata": {
   "formatVersion": 1
  },
  "creationString": "access_pattern_hin
...


El tamaño de almacenamiento de 10 documentos de 1 millón de registros con un peso en total en bruto de 2.41 gb (en escala base binario $2^{20}$), una vez comprimido en la base de datos es **0.93 gb** (en escala base binario $2^{20}$). Esto nos da un radio de compresión de 0.3868 y una reducción de tamaño en memoria del 61.32%.

![Cargando 1 millón de documentos](../imagenes-de-soporte/mongo-insert/storage_compass_10M_documentos_no_index.png)

Mediante la herramienta gráfica compass se puede ver un reporte de 1.00 GB (en escala base decimal $10^{6}$) de tamaño de almacenamiento, lo cual convertido a escala binaria resulta en 93.13 mb, un número aproximado al obtenido mediante el script de python.

In [48]:
print("...")
print(r[109935:])

...
 },
 "indexBuilds": [],
 "totalIndexSize": 1226719232,
 "totalSize": 2231209984,
 "indexSizes": {
  "_id_": 110129152,
  "fecha_1": 186114048,
  "agencia_1": 53514240,
  "monto_1": 158367744,
  "saldo_1": 173264896,
  "descripcion_1": 55648256,
  "nota_1": 489680896
 },
 "scaleFactor": 1,
 "ok": 1.0
}


Otro número importante que se meustra en compass es el tamaño total de los índices (7 índices en este caso), siendo 1.23 GB (en escala base decimal $10^{6}$) o según los datos extraídos mediante python **1.14 gb** (en escala base binario $2^{20}$).

Una última métrica importante ha revisar es la suma del espacio asignado tanto a los documentos como a los índices en la colección. Es la suma de storageSize y indexSize. En el presente caso resultando un valor de **2.08 gb**.



| Tabla resumen de resultados                 | Valor obtenido |
|---------------------------------------------|----------------|
| Tiempo de carga                             | 7 min 1.82 s   |
| Tamaño JSON bruto                           | 2.41 gb        |
| Tamaño de almacenamiento en mongoDB         | 0.93 gb        |
| Radio de compresión de documento            | 0.3868         |
| Reducción de tamaño de memoria de documento | 61.32 %        |
| Tamaño total de los índices                 | 1.14 gb        |
| Tamaño total de almacenamiento colección    | 2.08 gb        |

# Inserción de 30 millones de documentos y posterior indexación

En esta sección se revisará la carga de 30 millones de documentos en una colección sin índices y el tamaño de almacenamiento de dicha cantidad de documentos una vez ya indexados.

## Carga de documento

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-mongo-no-index.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-mongo-no-index.py -n 30

```

El script antes mencionado recibe como entrada la cantidad de documentos que cargará a la base de datos. Secuencialmente, el código creará la nueva colección donde se cargarán los datos, cargará la información almacenada en un documento JSON de forma masiva, y creará finalmente los índices correspondientes a todos los campos.

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/mongo-insert/load_30M_documentos_mongo.png)

Como se puede ver en la imagen superior, cargar 30 millones de documentos sobre campos y posteriormente indexar todos los campos ha tomado un tiempo de **37 minutos 31.62 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en la colección ```extracto_cuenta_30m``` de la siguiente manera.

In [ ]:
cnt = db.extracto_cuenta_30m.count_documents({})
print(f"Actualmente existen {cnt} documentos en 'extracto_cuenta_30m'")

Actualmente existen 30000000 documentos en 'extracto_cuenta_30m'


## Tamaño de almacenamiento de datos

Para encontrar el tamaño de los archivos JSON en su forma bruta, antes de cargarse a la base de datos, se utilizará el siguiente script:

In [50]:
file_sizes = []
file_names = []
sum = 0

n = 30

for i, json_file in enumerate(sorted(os.listdir("../json-generated-data"))):
    file_stat = os.stat(os.path.join("../json-generated-data", json_file))
    file_sizes.append(file_stat)
    file_names.append(json_file)
    sum = sum + int(file_stat.st_size)
    
    if i+1 == n:
        break

for file, f_name in zip(file_sizes, file_names):
    print(f'Documento {f_name} con tamaño en memoria de {file.st_size} bytes')

print("")
print(f'Sumando un total de {sum} bytes')

Documento random-generated-data-1.json con tamaño en memoria de 259734499 bytes
Documento random-generated-data-10.json con tamaño en memoria de 259687666 bytes
Documento random-generated-data-11.json con tamaño en memoria de 259697531 bytes
Documento random-generated-data-12.json con tamaño en memoria de 259660386 bytes
Documento random-generated-data-13.json con tamaño en memoria de 259651170 bytes
Documento random-generated-data-14.json con tamaño en memoria de 259696780 bytes
Documento random-generated-data-15.json con tamaño en memoria de 259771941 bytes
Documento random-generated-data-16.json con tamaño en memoria de 259765264 bytes
Documento random-generated-data-17.json con tamaño en memoria de 259736398 bytes
Documento random-generated-data-18.json con tamaño en memoria de 259722533 bytes
Documento random-generated-data-19.json con tamaño en memoria de 259695669 bytes
Documento random-generated-data-2.json con tamaño en memoria de 259739233 bytes
Documento random-generated-dat

Para obtener las estadísticas de uso de disco por parte de la colección ```extracto_cuenta_30m2``` en la base de datos ejecutaremos el siguiente comando:

In [52]:
r = dumps(db.command(
   {
     "collStats": "extracto_cuenta_30m",
     "scale": 1
   }
), indent=1)

print(r[:300])
print("...")


{
 "ns": "test_mongo.extracto_cuenta_30m",
 "size": 6182911222,
 "count": 30000000,
 "avgObjSize": 206,
 "numOrphanDocs": 0,
 "storageSize": 2984677376,
 "freeStorageSize": 954368,
 "capped": false,
 "wiredTiger": {
  "metadata": {
   "formatVersion": 1
  },
  "creationString": "access_pattern_hint=
...


El tamaño de almacenamiento de 30 documentos de 1 millón de registros con un peso en total en bruto de 7.26 gb (en escala base binario $2^{20}$), una vez comprimido en la base de datos es **2.78 gb** (en escala base binario $2^{20}$). Esto nos da un radio de compresión de 0.3830 y una reducción de tamaño en memoria del 61.69%.

![Cargando 1 millón de documentos](../imagenes-de-soporte/mongo-insert/storage_compass_30M_documentos_no_index.png)

Mediante la herramienta gráfica compass se puede ver un reporte de 2.98 GB (en escala base decimal $10^{6}$) de tamaño de almacenamiento, lo cual convertido a escala binaria resulta en 2.78 gb, un número aproximado al obtenido mediante el script de python.

In [56]:
print("...")
print(r[109750:])

...

 },
 "indexBuilds": [],
 "totalIndexSize": 3570143232,
 "totalSize": 6554820608,
 "indexSizes": {
  "_id_": 331038720,
  "fecha_1": 512319488,
  "agencia_1": 160526336,
  "monto_1": 468492288,
  "saldo_1": 493805568,
  "descripcion_1": 166879232,
  "nota_1": 1437081600
 },
 "scaleFactor": 1,
 "ok": 1.0
}


Otro número importante que se meustra en compass es el tamaño total de los índices (7 índices en este caso), siendo 3.32 GB (en escala base decimal $10^{9}$) o según los datos extraídos mediante python **3.32 gb** (en escala base binario $2^{20}$).

Una última métrica importante ha revisar es la suma del espacio asignado tanto a los documentos como a los índices en la colección. Es la suma de storageSize y indexSize. En el presente caso resultando un valor de **6.10 gb**.



| Tabla resumen de resultados                 | Valor obtenido |
|---------------------------------------------|----------------|
| Tiempo de carga                             | 37 min 31.62 s |
| Tamaño JSON bruto                           | 7.26 gb        |
| Tamaño de almacenamiento en mongoDB         | 2.68 gb        |
| Radio de compresión de documento            | 0.3830         |
| Reducción de tamaño de memoria de documento | 61.69 %        |
| Tamaño total de los índices                 | 3.32 gb        |
| Tamaño total de almacenamiento colección    | 6.10 gb        |